In [6]:
from asyncore import read
import random

# edit settings
READ_LENGTH = 20
READ_DISTANCE = READ_LENGTH
ERROR_RATE = 0.03
SHIFT_MIN = 1
SHIFT_MAX = 2
SHUFFLE_READS = True
REF_LENGTH = 1000
VARIANT_SIZE = random.randint(READ_LENGTH*3, READ_LENGTH*5)
VARIANT_PADDING = READ_LENGTH*2
# 0: Deletion
# 1: Insertion
# 2: Tandem Duplication
# 3: Inversion
# 4: Translocation
MODE = 0

# edit save file name
readsPath = 'read'
anwserPath = 'anwser'
refPath = 'ref'
INDEX = '01'
readsPath += '_'+INDEX
anwserPath += '_'+INDEX
refPath += '_'+INDEX

# Generate reference & variance
def generateReference(length):
    BASES = ['A','C','G','T']
    output = ''
    for i in range(length):
        output += random.choice(BASES)
    return output

ref = generateReference(REF_LENGTH)
index = random.randint(VARIANT_PADDING, len(ref)-VARIANT_PADDING-1)
anwser = ''
if MODE == 0:
    anwser += 'Deletion at index ' + str(index) + '\n'
    anwser += 'With size ' + str(VARIANT_SIZE) + '\n'
    anwser += ref[0:index] + ref[index+VARIANT_SIZE:]
elif MODE == 1:
    anwser += 'Insertion at index ' + str(index) + '\n'
    anwser += 'With size ' + str(VARIANT_SIZE) + '\n'
    anwser += ref[0:index] + generateReference(VARIANT_SIZE) + ref[index:]
elif MODE == 2:
    index = random.randint(VARIANT_PADDING+VARIANT_SIZE, len(ref)-VARIANT_PADDING-1)
    anwser += 'Tandem Duplication at index ' + str(index) + '\n'
    anwser += 'With size ' + str(VARIANT_SIZE) + '\n'
    anwser += ref[0:index] + ref[index-VARIANT_SIZE:index] + ref[index:]
elif MODE == 3:
    anwser += 'Inversion at index ' + str(index) + '\n'
    anwser += 'With size ' + str(VARIANT_SIZE) + '\n'
    anwser += ref[0:index] + ref[index:index+VARIANT_SIZE][::-1] + ref[index+VARIANT_SIZE:]
else:
    index = len(ref)-VARIANT_SIZE
    anwser += 'Translocation at index ' + str(index) + '\n'
    anwser += 'With size ' + str(VARIANT_SIZE) + '\n'
    anwser += ref[0:index] + generateReference(VARIANT_SIZE)


anwser += '\n' + ref + '\n'
count = 1
for i in range(max(len(anwser.split('\n')[2]),len(anwser.split('\n')[3]))):
    if (i%5 == 4):
        anwser+=str(count)
        count+=1
        count%=10
    else:
        anwser+=' '
anwser += '\n' + ' '*index + '^'
anwserSplit = anwser.split('\n')
anwser = '\n'.join([anwserSplit[0],anwserSplit[1],anwserSplit[4],anwserSplit[3],anwserSplit[2],anwserSplit[5]])

# Generate reads
reads = []
physical = anwser.split('\n')[4]

def generateRead(index):
    global reads
    global anwser
    NEG_BASES = {'A':['C','G','T'],'C':['A','G','T'],'G':['A','C','T'],'T':['A','C','G']}

    read1 = ''
    for i in range(index,index+READ_LENGTH):
        if random.random() < ERROR_RATE:
            read1 += random.choice(NEG_BASES[physical[i]])
        else:
            read1 += physical[i]

    read2 = ''
    for i in range(index+READ_LENGTH+READ_DISTANCE,index+READ_LENGTH*2+READ_DISTANCE):
        if random.random() < ERROR_RATE:
            read2 += random.choice(NEG_BASES[physical[i]])
        else:
            read2 += physical[i]
    
    reads.append([read1,read2])
    line = ' ' * index + read1 + ' ' * READ_DISTANCE + read2
    anwser += '\n' + line

index = 0
while index < len(physical) - 2*READ_LENGTH - READ_DISTANCE - 1:
    generateRead(index)
    index += random.randint(SHIFT_MIN, SHIFT_MAX)

index = len(physical) - 2*READ_LENGTH - READ_DISTANCE
generateRead(index)

if SHUFFLE_READS:
    random.shuffle(reads)

readsText = '\n'.join(','.join(row) for row in reads)

print('-----------{} reads-----------'.format(len(reads)))
#print(readsText)
print('\n-----------Anwser-----------')
#print(anwser)
print('\n'.join(anwser.split('\n')[:6]))

# Save files
if readsPath:
    with open(readsPath+'.txt','w') as f:
        f.write(readsText)
if anwserPath:
    with open(anwserPath+'.txt','w') as f:
        f.write(anwser)
if refPath:
    with open(refPath+'.txt','w') as f:
        f.write(ref)


-----------1234 reads-----------

-----------Anwser-----------
Deletion at index 827
With size 80
    1    2    3    4    5    6    7    8    9    0    1    2    3    4    5    6    7    8    9    0    1    2    3    4    5    6    7    8    9    0    1    2    3    4    5    6    7    8    9    0    1    2    3    4    5    6    7    8    9    0    1    2    3    4    5    6    7    8    9    0    1    2    3    4    5    6    7    8    9    0    1    2    3    4    5    6    7    8    9    0    1    2    3    4    5    6    7    8    9    0    1    2    3    4    5    6    7    8    9    0    1    2    3    4    5    6    7    8    9    0    1    2    3    4    5    6    7    8    9    0    1    2    3    4    5    6    7    8    9    0    1    2    3    4    5    6    7    8    9    0    1    2    3    4    5    6    7    8    9    0    1    2    3    4    5    6    7    8    9    0    1    2    3    4    5    6    7    8    9    0    1    2    3    4    5    6    7    8    9    0  